## Gaia ESA Archive

![](https://gea.esac.esa.int/archive-help/css/images/header-cosmos-gaia.png)

Reference: https://gea.esac.esa.int/archive/

In [ ]:
''' Install astropy '''
!pip install astropy

In [ ]:
from bioexplorer import BioExplorer, Vector3
import requests
import gzip
import astropy.coordinates as coord
import astropy.units as u
from tqdm import tqdm
import os

'''Connect to the BioExplorer and reset scene'''
url = 'localhost:5000'
be = BioExplorer(url)
status = be.reset_scene()

'''Default settings'''
DEFAULT_RADIUS = 0.00005

'''Gaia data source'''
url = "http://cdn.gea.esac.esa.int/Gaia/gdr1/gaia_source/csv/"
cache_folder = '/tmp'

a = 0
for b in range(21):
    for c in range(256):
        positions = list()
        radii = list()

        '''Download file if not already in the cache'''
        base_name = 'GaiaSource_%03d-%03d-%03d' % (a, b, c)
        path = os.path.join(cache_folder, base_name + '.csv.gz')
        lines = list()
        if not os.path.exists(path):
            response = requests.get(url + base_name + '.csv.gz')
            content = response.content

            # Save the file
            with open(path, 'wb') as file:
                file.write(content)
            lines = gzip.decompress(content).decode('utf-8').split('\n')
        else:
            lines = gzip.open(path).readlines()

        header = True
        for line in tqdm(lines):
            if header:
                header = False
                continue
            values = str(line).split(',')

            '''Convert the RA and DEC coordinates to 3D coordinates'''
            ra = float(values[4])
            dec = float(values[6])
            icrs = coord.SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame='icrs')
            galactic = icrs.transform_to(coord.Galactic)

            '''Extract the x, y, z coordinates'''
            x = float(galactic.cartesian.x)
            y = float(galactic.cartesian.y)
            z = float(galactic.cartesian.z)
            
            '''Add coordinates and default radius to the list of positions'''
            positions.append(Vector3(x, y, z))
            radii.append(DEFAULT_RADIUS)

        '''Add spheres to the BioExplorer'''
        status = be.add_spheres(
            name=base_name,
            positions=positions, radii=radii)